In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch as tc
import cupy as cp

from model.ffnn import FFNN

tc.autograd.set_grad_enabled(False)
        
if tc.cuda.is_available():
    print("CUDA tersedia!")
else:
    print("CUDA tidak tersedia!")
    

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
tc.set_default_device(device)
print("Using CUDA device:", tc.cuda.get_device_name(device))


# cp.cuda.Device(0).use()




CUDA tersedia!
Using CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype(np.float64)
y = mnist.target.astype(np.int32)


X = X / 255.0
# print(X)
print("Dataset MNIST_784 dimuat: X shape:", X.shape, " y shape:", y.shape)

Dataset MNIST_784 dimuat: X shape: (70000, 784)  y shape: (70000,)


In [3]:

subset_size = 70000
indices = np.random.choice(np.arange(X.shape[0]), subset_size, replace=False)
X_subset = X.iloc[indices]
y_subset = y.iloc[indices]

X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)


print("Data subset: Training:", X_train.shape, "Testing:", X_test.shape)

Data subset: Training: (56000, 784) Testing: (14000, 784)


In [4]:
def one_hot(labels, num_classes):
    oh = np.zeros((labels.shape[0], num_classes))
    oh[np.arange(labels.shape[0]), labels] = 1.0
    return oh

num_classes = 10
y_train_oh = one_hot(y_train, num_classes)
y_test_oh = one_hot(y_test, num_classes)

In [5]:
mlp = MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=100000, random_state=42, learning_rate_init=0.001, batch_size=200, verbose=True)

start_time = time.time()
mlp.fit(X_train, y_train)
sklearn_training_time = time.time() - start_time

#prediksi
y_pred_sklearn = mlp.predict(X_test)
accuracy_sklearn = np.mean(y_pred_sklearn == y_test)
print("Akurasi MLPClassifier:", accuracy_sklearn)
print("Waktu training MLPClassifier: {:.2f} detik".format(sklearn_training_time))

Iteration 1, loss = 0.40853828
Iteration 2, loss = 0.15944046
Iteration 3, loss = 0.11339504
Iteration 4, loss = 0.08640177
Iteration 5, loss = 0.06774480
Iteration 6, loss = 0.05679317
Iteration 7, loss = 0.04620483
Iteration 8, loss = 0.03976023
Iteration 9, loss = 0.03177209
Iteration 10, loss = 0.02586417
Iteration 11, loss = 0.02227854
Iteration 12, loss = 0.02120353
Iteration 13, loss = 0.01924831
Iteration 14, loss = 0.01339245
Iteration 15, loss = 0.01262281
Iteration 16, loss = 0.01333962
Iteration 17, loss = 0.01125821
Iteration 18, loss = 0.01063192
Iteration 19, loss = 0.01169848
Iteration 20, loss = 0.00986919
Iteration 21, loss = 0.00950743
Iteration 22, loss = 0.01072083
Iteration 23, loss = 0.00575859
Iteration 24, loss = 0.00720443
Iteration 25, loss = 0.00495888
Iteration 26, loss = 0.00708670
Iteration 27, loss = 0.00827014
Iteration 28, loss = 0.00551774
Iteration 29, loss = 0.00577067
Iteration 30, loss = 0.01212517
Iteration 31, loss = 0.00752265
Iteration 32, los

In [6]:
# X_train = cp.asarray(X_train)
# y_train_oh = cp.asarray(y_train_oh)
# X_test = cp.asarray(X_test)
# y_test = cp.asarray(y_test)

# custom_model = FFNN(
#     [784, 64, 10],
#     activations_list=['relu', 'sigmoid'],
#     loss_function='mse',  
#     weight_init='random_uniform',
#     init_params={'lower': -1, 'upper': 1}
# )

# epochs_custom = 10000  
# batch_size_custom = 200
# lr_custom = 0.001

# start_time = time.time()
# history_custom = custom_model.train(X_train, y_train_oh, epochs=epochs_custom, batch_size=batch_size_custom, learning_rate=lr_custom, verbose=False)
# custom_training_time = time.time() - start_time
# print(f"Model custom selesai training dalam {custom_training_time:.2f} detik.")

# # Prediksi pada data test
# y_pred_custom = custom_model.forward(X_test)
# # y_pred_custom.data adalah NumPy array, sehingga gunakan np.argmax dengan parameter axis
# y_pred_custom_labels = cp.argmax(y_pred_custom, axis=1)
# accuracy_custom = cp.mean(y_pred_custom_labels == y_test)
# print("Akurasi model custom FFNN:", accuracy_custom)

In [ ]:


X_train = tc.tensor(np.asarray(X_train), dtype=tc.float32)
X_test = tc.tensor(np.asarray(X_test), dtype=tc.float32)
y_train_oh = tc.tensor(np.asarray(y_train_oh), dtype=tc.float32)
y_test = tc.tensor(np.asarray(y_test), dtype=tc.int64)

print(X_train.get_device())

# Inisialisasi model custom FFNN
custom_model = FFNN(
    [784, 256, 128, 64, 10],
    activations_list=['relu', 'relu', 'relu', 'softmax'],
    loss_function='cce',  
    weight_init='he_xavier',
    init_params={'lower': -1, 'upper': 1}
)
epochs_custom = 100000
batch_size_custom = 200
lr_custom = 0.001

start_time = time.time()
history_custom = custom_model.train(X_train, y_train_oh, epochs=epochs_custom, batch_size=batch_size_custom, learning_rate=lr_custom, verbose=True)
custom_training_time = time.time() - start_time
print(f"Model custom selesai training dalam {custom_training_time:.2f} detik.")

# Prediksi 
y_pred_custom = custom_model.forward(X_test)
y_pred_custom_labels = tc.argmax(y_pred_custom.data, dim=1)
accuracy_custom = (y_pred_custom_labels == y_test).float().mean().item()
print("Akurasi model custom FFNN:", accuracy_custom)


#107,32s tc cpu
#158,16s tc gpu
#215,52s cp gpu
#230,48s np cpu

#555.06 detik

#4000 20m

# Model custom selesai training dalam 1135.40 detik. GPU
# Akurasi model custom FFNN: 0.9748571515083313

# Model custom selesai training dalam 1193.06 detik. CPU
# Akurasi model custom FFNN: 0.9740714430809021

# Model custom selesai training dalam 733.88 detik.
# Akurasi model custom FFNN: 0.9735714197158813


0
Epoch 1/100000 - Train Loss: 2.29328291
Epoch 2/100000 - Train Loss: 2.06913726
Epoch 3/100000 - Train Loss: 1.84946939
Epoch 4/100000 - Train Loss: 1.61635886
Epoch 5/100000 - Train Loss: 1.38599320
Epoch 6/100000 - Train Loss: 1.18285971
Epoch 7/100000 - Train Loss: 1.01915672
Epoch 8/100000 - Train Loss: 0.89366738
Epoch 9/100000 - Train Loss: 0.79865181
Epoch 10/100000 - Train Loss: 0.72561795
Epoch 11/100000 - Train Loss: 0.66815051
Epoch 12/100000 - Train Loss: 0.62185101
Epoch 13/100000 - Train Loss: 0.58380330
Epoch 14/100000 - Train Loss: 0.55203352
Epoch 15/100000 - Train Loss: 0.52524581
Epoch 16/100000 - Train Loss: 0.50233176
Epoch 17/100000 - Train Loss: 0.48255585
Epoch 18/100000 - Train Loss: 0.46530987
Epoch 19/100000 - Train Loss: 0.45012938
Epoch 20/100000 - Train Loss: 0.43671998
Epoch 21/100000 - Train Loss: 0.42469192
Epoch 22/100000 - Train Loss: 0.41395158
Epoch 23/100000 - Train Loss: 0.40426377
Epoch 24/100000 - Train Loss: 0.39538600
Epoch 25/100000 - Train